In [ ]:
import datetime
import panel as pn

pn.extension('tabulator', notifications=True)

import panel as pn
import matplotlib.pyplot as plt
from src.dao import produtor_dao, plantio_dao, propriedade_dao

pn.extension('tabulator', notifications=True)

# Título do Sistema
titulo = pn.pane.Markdown("# 🚜 AgroGestão - Sistema Integrado")

In [ ]:
# --- WIDGETS (CAMPOS DA TELA) ---
# ID Oculto (para saber se é edição)
w_id = pn.widgets.IntInput(name="ID", disabled=True, visible=False)

w_nome = pn.widgets.TextInput(name="Nome Completo", placeholder="Ex: João da Silva")
w_cpf = pn.widgets.TextInput(name="CPF", placeholder="12345678900")
w_nasc = pn.widgets.DatePicker(name="Data de Nascimento")
w_insc = pn.widgets.TextInput(name="Inscrição Estadual", placeholder="Ex: 200500-1")
w_busca = pn.widgets.TextInput(name="🔍 Buscar Produtor", placeholder="Digite o nome e dê Enter")

# Botões
btn_salvar = pn.widgets.Button(name='💾 Salvar', button_type='success')
btn_limpar = pn.widgets.Button(name='🧹 Limpar', button_type='default')
btn_excluir = pn.widgets.Button(name='🗑️ Excluir', button_type='danger', disabled=True)

# Tabela
tabela = pn.widgets.Tabulator(show_index=False, sizing_mode='stretch_width', page_size=10)

# --- LÓGICA (EVENTOS) ---

async def carregar_produtores(event=None):
    """Busca dados no banco e atualiza a tabela"""
    df = await produtor_dao.listar(w_busca.value)
    tabela.value = df
    # Atualiza gráfico também
    await atualizar_grafico()

def preencher_formulario(event):
    # 1. Verifica se o clique foi válido
    if event.row is None:
        return

    try:
        # CORREÇÃO: Usamos o número da linha (event.row) para buscar os dados no DataFrame da tabela
        row_index = event.row
        df = tabela.value # O DataFrame atual da tabela
        
        # Pega a linha correta usando iloc (pela posição)
        row = df.iloc[row_index]
        
        print(f"✅ Dados encontrados: {row.to_dict()}") # Debug para confirmar

        # 2. Preenchimento dos Campos
        # Atenção: Os nomes aqui ('id_pessoa', 'nome', etc) devem ser iguais aos do banco!
        
        # ID (Chave Primária)
        if 'id_pessoa' in row:
            w_id.value = int(row['id_pessoa'])
        else:
            # Fallback caso a coluna se chame 'id'
            w_id.value = int(row['id'])

        w_nome.value = str(row['nome'])
        w_cpf.value = str(row['cpf'])
        w_insc.value = str(row['inscricao_estadual'])
        
        # Tratamento de Data
        if 'data_nascimento' in row and row['data_nascimento']:
            val = row['data_nascimento']
            # Se for timestamp ou datetime, converte para date
            if hasattr(val, 'date'):
                w_nasc.value = val.date()
            else:
                w_nasc.value = val # Assume que já é date ou string compatível
        
        # 3. Atualizar Estado dos Botões
        btn_salvar.name = "🔄 Atualizar"
        btn_salvar.button_type = "warning"
        btn_excluir.disabled = False
        
        pn.state.notifications.info(f"Editando: {row['nome']}")

    except Exception as e:
        msg = f"❌ Erro ao processar linha: {str(e)}"
        print(msg)
        pn.state.notifications.error(msg)
tabela.on_click(preencher_formulario)

async def salvar_clique(event):
    try:
        btn_salvar.loading = True
        
        # Validação simples
        if not w_cpf.value or not w_nome.value:
            pn.state.notifications.error("Nome e CPF são obrigatórios!")
            return

        if w_id.value: # TEM ID? ENTÃO É ATUALIZAÇÃO
            await produtor_dao.atualizar(w_id.value, w_nome.value, w_cpf.value, w_nasc.value, w_insc.value)
            pn.state.notifications.success("Produtor atualizado!")
        else: # NÃO TEM ID? ENTÃO É INSERÇÃO
            await produtor_dao.inserir(w_nome.value, w_cpf.value, w_nasc.value, w_insc.value)
            pn.state.notifications.success("Produtor cadastrado!")
        
        limpar_clique(None) # Reseta formulário
        await carregar_produtores() # Recarrega lista
        
    except Exception as e:
        pn.state.notifications.error(f"Erro: {str(e)}")
    finally:
        btn_salvar.loading = False

async def excluir_clique(event):
    if w_id.value:
        try:
            await produtor_dao.excluir(w_id.value)
            pn.state.notifications.success("Produtor excluído!")
            limpar_clique(None)
            await carregar_produtores()
        except Exception as e:
            pn.state.notifications.error(f"Erro ao excluir: {e}")

def limpar_clique(event):
    w_id.value = 0 # Reseta ID
    w_nome.value = ""
    w_cpf.value = ""
    w_insc.value = ""
    w_nasc.value = None
    
    # Reseta botões
    btn_salvar.name = "💾 Salvar"
    btn_salvar.button_type = "success"
    btn_excluir.disabled = True

# Ligar botões às funções
btn_salvar.on_click(salvar_clique)
btn_limpar.on_click(limpar_clique)
btn_excluir.on_click(excluir_clique)
w_busca.param.watch(carregar_produtores, 'value') # Busca ao digitar

# --- GRÁFICO ---
pane_grafico = pn.pane.Matplotlib(tight=True)

async def atualizar_grafico():
    df = await produtor_dao.dados_grafico_idade()
    if df.empty: return
    
    fig, ax = plt.subplots(figsize=(5, 3))
    ax.bar(df['faixa_etaria'], df['total'], color='#4CAF50')
    ax.set_title("Produtores por Faixa Etária")
    ax.set_ylabel("Quantidade")
    plt.close(fig)
    pane_grafico.object = fig

# --- LAYOUT FINAL ---
pn.state.onload(carregar_produtores) # Carrega ao abrir

layout_produtor = pn.Column(
    pn.pane.Markdown("# 🚜 Módulo Produtor"),
    pn.Row(
        pn.Column(
            pn.pane.Markdown("### Cadastro"),
            w_id, w_nome, w_cpf, w_nasc, w_insc,
            pn.Row(btn_salvar, btn_limpar, btn_excluir)
        ),
        pn.Column(
            pn.pane.Markdown("### Listagem"),
            w_busca,
            tabela,
            pn.pane.Markdown("### Estatísticas"),
            pane_grafico
        )
    )
)

In [ ]:
# --- WIDGETS ---
w_plant_id = pn.widgets.IntInput(name="ID", disabled=True, visible=False)

# Selects (Carregados do Banco)
w_sel_safra = pn.widgets.Select(name="Safra")
w_sel_cultivo = pn.widgets.Select(name="Cultivo/Variedade")
w_sel_prop = pn.widgets.Select(name="Propriedade")

w_data_plantio = pn.widgets.DatePicker(name="Data do Plantio", value=datetime.date.today())
w_area_plantada = pn.widgets.FloatInput(name="Área Plantada (Hectares)", value=0.0, step=0.1)

btn_plant_salvar = pn.widgets.Button(name='🌱 Registrar Plantio', button_type='success')
btn_plant_limpar = pn.widgets.Button(name='🧹 Limpar', button_type='default')
btn_plant_excluir = pn.widgets.Button(name='🗑️ Excluir', button_type='danger', disabled=True)

tab_plant = pn.widgets.Tabulator(show_index=False, sizing_mode='stretch_width', page_size=5)
graf_plant = pn.pane.Matplotlib(tight=True)

# --- LÓGICA ---

async def carregar_opcoes_plantio():
    """Preenche os menus com dados do banco"""
    w_sel_safra.options = await plantio_dao.get_opcoes_safra()
    w_sel_cultivo.options = await plantio_dao.get_opcoes_cultivo()
    w_sel_prop.options = await plantio_dao.get_opcoes_propriedade()

async def carregar_plantios(event=None):
    # 1. Carrega opções (para garantir que estão atualizadas)
    await carregar_opcoes_plantio()
    
    # 2. Carrega Tabela
    tab_plant.value = await plantio_dao.listar()
    
    # 3. Carrega Gráfico
    df_g = await plantio_dao.dados_grafico_cultura()
    if not df_g.empty:
        fig, ax = plt.subplots(figsize=(5, 3))
        ax.pie(df_g['Total Hectares'], labels=df_g['Cultura'], autopct='%1.1f%%', startangle=90, colors=['#8dd3c7','#ffffb3','#bebada','#fb8072'])
        ax.set_title("Área Plantada por Cultura")
        plt.close(fig)
        graf_plant.object = fig

def selecionar_plantio(event):
    # 1. Verificação de segurança
    if event.row is None:
        return

    try:
        # CORREÇÃO: Usar o índice para buscar os dados no DataFrame da tabela
        row_index = event.row
        df = tab_plant.value # Pega o DataFrame atual da tabela
        row = df.iloc[row_index] # Acessa a linha correta
        
        print(f"✅ Dados Plantio selecionados: {row.to_dict()}")

        # 2. Preencher os Campos (Baseado nos nomes das colunas do plantio_dao)
        # Colunas esperadas: 'id', 'Safra', 'Cultura', 'Propriedade', 'Data Plantio', 'Área (ha)'
        
        w_plant_id.value = int(row['id'])
        
        # Área (tem de converter para float para garantir)
        if 'Área (ha)' in row:
            w_area_plantada.value = float(row['Área (ha)'])
            
        # Data
        if 'Data Plantio' in row and row['Data Plantio']:
            val = row['Data Plantio']
            if hasattr(val, 'date'):
                w_data_plantio.value = val.date()
            else:
                w_data_plantio.value = val # Tenta atribuir direto se já for date
        
        # 3. Atualizar Estado dos Botões
        btn_plant_salvar.name = "🔄 Atualizar Dados"
        btn_plant_salvar.button_type = "warning"
        btn_plant_excluir.disabled = False
        
        pn.state.notifications.info(f"Editando Plantio #{row['id']}")

    except Exception as e:
        msg = f"❌ Erro ao selecionar plantio: {str(e)}"
        print(msg)
        pn.state.notifications.error(msg)

async def salvar_plantio(event):
    try:
        if not w_sel_safra.value or not w_sel_cultivo.value or not w_sel_prop.value:
            pn.state.notifications.error("Selecione Safra, Cultivo e Propriedade!")
            return
        if w_area_plantada.value <= 0:
            pn.state.notifications.error("A área deve ser maior que zero!")
            return

        btn_plant_salvar.loading = True
        
        if w_plant_id.value: # Atualizar
            await plantio_dao.atualizar(
                w_plant_id.value, w_sel_safra.value, w_sel_cultivo.value, 
                w_sel_prop.value, w_data_plantio.value, w_area_plantada.value
            )
            pn.state.notifications.success("Plantio atualizado!")
        else: # Inserir
            await plantio_dao.inserir(
                w_sel_safra.value, w_sel_cultivo.value, 
                w_sel_prop.value, w_data_plantio.value, w_area_plantada.value
            )
            pn.state.notifications.success("Plantio registrado!")

        limpar_plantio(None)
        await carregar_plantios()
    except Exception as e:
        pn.state.notifications.error(f"Erro: {e}")
    finally:
        btn_plant_salvar.loading = False

async def excluir_plantio(event):
    if w_plant_id.value:
        try:
            await plantio_dao.excluir(w_plant_id.value)
            pn.state.notifications.success("Plantio excluído!")
            limpar_plantio(None)
            await carregar_plantios()
        except Exception as e:
            pn.state.notifications.error(f"Erro ao excluir: {e}")

def limpar_plantio(event):
    w_plant_id.value = 0
    w_data_plantio.value = datetime.date.today()
    w_area_plantada.value = 0.0
    btn_plant_salvar.name = "🌱 Registrar Plantio"
    btn_plant_salvar.button_type = "success"
    btn_plant_excluir.disabled = True

# --- LIGAÇÕES ---
btn_plant_salvar.on_click(salvar_plantio)
btn_plant_limpar.on_click(limpar_plantio)
btn_plant_excluir.on_click(excluir_plantio)
tab_plant.on_click(selecionar_plantio)

# --- LAYOUT ---
layout_plantio = pn.Row(
    pn.Column(
        "### 🚜 Novo Plantio", 
        w_sel_safra, 
        w_sel_cultivo, 
        w_sel_prop, 
        w_data_plantio, 
        w_area_plantada, 
        pn.Row(btn_plant_salvar, btn_plant_limpar, btn_plant_excluir)
    ),
    pn.Column(
        "### Histórico de Plantios", 
        tab_plant, 
        "### Estatísticas (Área)",
        graf_plant
    )
)

In [ ]:
# CÉLULA 5 - DASHBOARD FINAL

# Função para carregar tudo ao abrir
async def inicializar_tudo():
    await carregar_produtores()
#    await carregar_propriedades()
    await carregar_plantios()

pn.state.onload(inicializar_tudo)

# Cria as Abas Navegáveis
dashboard = pn.Tabs(
    ("🧑‍🌾 Produtores", layout_produtor),
 #   ("🏡 Propriedades", layout_propriedade),
    ("🌱 Plantios", layout_plantio)
)

# Exibe o Dashboard Completo
pn.Column(
    titulo,
    dashboard
).servable()